# Plotting and smoothing taxi data

## Introduction

I chose to analyze the T-Drive trajectory data sample (https://www.microsoft.com/en-us/research/publication/t-drive-trajectory-data-sample/). This dataset contains the GPS trajectories for taxis in the Beijing area from February 2, 2008, to February 8, 2008<sup>1,2</sup>. 

The first part of the task was to produce the first two figures from the document summarizing the dataset (User_guide_T-drive.pdf) found at the site above.  The figures show histograms of the distance and time intervals between two consecutive points). The second part of the task was to present a smoothed trajectory using spline or Kalman filters of one of the taxis.

The dataset can be found at the above site (under the tag "Related Info") as 9 zipped files ("06","07","08","09","010","011","012","013","014").  Each zipped file contains approximately 1000 txt files named as  taxi_id.txt, where taxi_id is a number identification from 1 to 10357. The file format (for each line) is:<br>
taxi id, date time, longitude, latitude<br>
separated by commas.

The code is written in R.

### References

[1] Jing Yuan, Yu Zheng, Xing Xie, and Guangzhong Sun. Driving with knowledge from the physical world. In The 17th ACM SIGKDD international conference on Knowledge Discovery and Data mining, KDD’11, New York, NY, USA, 2011. ACM.<br>
[2] Jing Yuan, Yu Zheng, Chengyang Zhang, Wenlei Xie, Xing Xie, Guangzhong Sun, and Yan Huang. T-drive: driving directions based on taxi trajectories. In Proceedings of the 18th SIGSPATIAL International Conference on Advances in Geographic Information Systems, GIS ’10, pages 99-108, New York, NY, USA,2010. ACM.

## getfilenames.R

The 9 zipped files were downloaded to the home directory and unzipped, creating 9 subdirectories: "06","07","08","09","010","011","012","013","014". These names were put in the vector *indir*.

This code goes through each directory and creates a dataframe *filelist* with 2 columns (for all the files in all the subdirectories). <br>
Column 1: an index (1-9) into the vector indir to access the the subdirectory name <br>
Column 2: the file name.<br>

*l* is a temporary dataframe holding the index and the file names for the subdirectory in the loop.<br>

Note that running this code found 8911 unique files across the dataset. This value is different than the value given in the documentation for this dataset (User_guide_T-drive.pdf), which listed 10,357 files ("10357" was the last taxi_id found).

In [3]:
# getfilenames.R

#home <- "C:/Users/ibshi/Desktop/startup.ml/taxi"
home <- "C:/Users/Steve S/Desktop/startup.ml/taxi"
setwd(home)

indir <- c("06","07","08","09","010","011","012","013","014")
num_indir <- length(indir)

# loop through each subdirectory

for (i in 1:num_indir) { 

    # temp holds the subdirectory name
    temp <- paste(home, indir[i], sep="/")
    
    # put file names into l
    l <- list.files(path=temp)
    
    # add an index for the subdirectory name vector indir to l
    index <- c(rep.int(i,length(l)))
    l <- cbind(index,l)
    
    # add l to filelist
    if (i == 1) {
        filelist <- l
    } 
    else {
      filelist <- rbind(filelist,l)
    }
    
} 


## readfiles.R

This code goes through the dataframe *filelist*, reads in the txt files with read.csv(), and creates then appends a dataframe *taxi_data* with the columns listed above <br>
taxi_id, date_time, longitude, latitude<br>

To handle any empty files, a tryCatch statement sets *t* to either the new data read in, or to a null dataframe *s*.

Also, a column was added, begin, which indicated the beginning of a new taxi_id, coded as 1/0 (yes/no), giving a final format for *taxi_data* of<br>
taxi_id, date_time, longitude, latitude, begin<br>
The begin column was added so that the distance and time intervals could be calcuated across all rows of *taxi_data* first, followed by setting interval values to NA for the beginning of a new taxi id. 

Each file was first read into a temporary dataframe *t*, which was then appended to *taxi_data*.

Note that readfiles.R took about 11 hours to run on a standard PC laptop (Intel Core i7-6700, 2.6 GHz, 8G RAM).  


In [ ]:
# readfiles.R

num_files <- nrow(filelist)
#num_files <- 100 # to test code for subsequent steps
print(paste("num_files = ",as.character(num_files)))

# s is a null dataframe for read.csv() failures
s <- data.frame(NULL,NULL,NULL,NULL)    

# give s column names that match a successful read.csv() result
s <- colnames(c("V1","V2","V3","V4")) 

# loop through filelist to read data into taxi_data

for (j in 1:num_files) {

  # tempname is the full name of a txt file

  t_ind <- as.integer(filelist[j,1]) 
  tempname <- paste(home,indir[t_ind],filelist[j,2],sep="/") 

  # a tryCatch that returns a temporary dataframe t with data from the tempdata file, or a null s dataframe

  t <- tryCatch(read.csv(tempname, header = FALSE),error = function(e) {print(paste("error retrieving",tempname));return(s);})

  # conditional upon t$V1[1] not being null, 2 steps were done: 
  # 1. add begin column to t to indicate a new taxi_id 
  # 2. add t to taxi_data  

  if (!is.null(t$V1[1])) { 
  
      # 1. add begin column to t to indicate a new taxi_id 
      
      t_len <- nrow(t)                
      t_begin <- c(rep(0,t_len))
      t_begin[1] <- 1
      t_begin <- as.factor(t_begin)
      t <- cbind(t,t_begin)   
      
      # 2. add t to taxi_data
      
      if (j == 1) {
        taxi_data <- t
      } 
      else {
        taxi_data <- rbind(taxi_data,t)
      }
  }

} 

colnames(taxi_data) <- c("taxi_id","date_time","longitude","latitude","begin")


### Results of readfiles.R

As stated above, readfiles.R took about 11 hours to run on a standard laptop. The output of the above code was: 

"num_files =  8911"<br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/10115.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/10352.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/1089.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/1497.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/1947.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/2929.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/2945.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/295.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/3050.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/3160.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/3194.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/3950.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/5972.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/6030.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/6236.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/6322.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/6717.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/7583.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/8209.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/8424.txt" <br>
[1] "error retrieving C:/Users/ibshi/Desktop/startup.ml/taxi/014/9874.txt" <br>

Thus, 21 files had read errors (due to empty files), and the final count of files was 8911 - 21 = 8890.<br>
The total number of rows for taxi_data points was 10,088,351, which is about 2/3 of the number stated in User_guide_T-drive.pdf (15 million).<br>


## intervals.R

This code adds a column interval_sec to *taxi_data* using the lubridate package. The steps are:<br>
1. add a column to taxi_data (new_date_time) converting the input date_time column to POSIXct format<br> 
2. add a column to taxi_data (new_date_time_m1) with the new_date_time of the previous row<br>
3. add a column to taxi_data (interval_sec) with the duration (in seconds) of the interval between new_date_time and new_date_time_m1. This was done across all rows; thus, the "interval" for the beginning of a new taxi_id is incorrect.<br>
4. set interval_sec for the beginning of a new taxi_id to NA.<br>
5. Drop the date_time and new_date_time_m1 columns (to save on RAM).<br>


In [ ]:
# intervals.R

library(lubridate)

# add a column (new_date_time), converting the input date_time column to POSIXct date time format
taxi_data$new_date_time <- ymd_hms(taxi_data$date_time)

# add a column (new_date_time_m1) with the new_date_time of the previous row 
num_datam1 <- nrow(taxi_data) - 1
t <- taxi_data$new_date_time[1:num_datam1]
t <- c(t[1],t)
taxi_data$new_date_time_m1 <- t
rm(t)

# add a column to taxi_data (interval_sec) with the duration (in seconds) of the interval between 
# new_date_time and new_date_time_m1. 
taxi_data$interval_sec <- as.duration(interval(taxi_data$new_date_time_m1,taxi_data$new_date_time))

# set interval_sec for the beginning of a new taxi_id to NA
taxi_data$interval_sec[which(taxi_data$begin == 1)] <- NA

# Drop the date_time and new_date_time_m1 columns
taxi_data <- subset.data.frame(taxi_data,select = c("taxi_id","longitude","latitude","begin","new_date_time","interval_sec"))


## lat_long_conv.R

This code first find the differences between adjacent times in latitude and longitude, and then converts this difference into distance, in meters.

1. Add columns lat_m1 and long_m1 to *taxi_data* for the latitudes and longitudes of the previous rows. <br>
2. Add columns diff_lat, diff_long, and ave_lat to *taxi_data* using latitude, lat_m1, longitude, and long_m1 <br>
3. remove intermediary columns lat_m1 and long_m1 from *taxi_data* <br>
4. Convert differences in latitude and longitude to distances in meters. This section uses the equations used by, for example: 
http://www.csgnetwork.com/degreelenllavcalc.html, which is referenced in Wikipedia (https://en.wikipedia.org/wiki/Geographic_coordinate_system), and is apparently accurate to 1 cm per degree latitude/longitude.<br>
  a. Add columns lat_meters_per_1deg and lat_meters_per_1deg to *taxi_data* computing the length of 1 degree latitude and  longitude for the average latitude between 2 adjacent points <br>
  b. Convert columns diff_lat and diff_long to meters <br>
  c. Add column distance (in meters) to *taxi_data* by computing Euclidean distance from columns diff_lat and diff_long <br>
  d. Set distance for the beginning of a new taxi_id to NA <br>
  e. Remove intermediary columns diff_lat and diff_long from  from *taxi_data* <br>


In [ ]:
# lat_long_conv.R

# Add columns lat_m1 and long_m1 to *taxi_data* for the latitudes and longitudes of the previous rows.

num_datam1 <- nrow(taxi_data) - 1
t <- taxi_data$latitude[1:num_datam1]
t <- c(t[1],t)
taxi_data$lat_m1 <- t

t <- taxi_data$longitude[1:num_datam1]
t <- c(t[1],t)
taxi_data$long_m1 <- t

rm(t)

# Add columns diff_lat, diff_long, and ave_lat to *taxi_data* using latitude, lat_m1, longitude, and long_m1 

taxi_data$diff_lat <- abs(taxi_data$latitude - taxi_data$lat_m1)
taxi_data$diff_long <- abs(taxi_data$longitude - taxi_data$long_m1)
taxi_data$ave_lat <- (taxi_data$latitude + taxi_data$lat_m1)/2

# remove intermediary columns lat_m1 and long_m1 

taxi_data <- subset.data.frame(taxi_data,select = c("taxi_id","longitude","latitude","begin","new_date_time","interval_sec","diff_lat","diff_long","ave_lat"))

# Convert differences in latitude and longitude to distances in meters. 

# this section uses the equations used by, for example:
# http://www.csgnetwork.com/degreelenllavcalc.html,
# which is referenced in Wikipedia (https://en.wikipedia.org/wiki/Geographic_coordinate_system),
# and is apparently accurate to 1 cm per degree latitude/longitude

m1 <- 111132.92     # latitude calculation term 1
m2 <- -559.82       # latitude calculation term 2
m3 <- 1.175         # latitude calculation term 3
m4 <- -0.0023       # latitude calculation term 4
p1 <- 111412.84     # longitude calculation term 1
p2 <- -93.5         # longitude calculation term 2
p3 <- -0.118         # longitude calculation term 3

  # Add columns lat_meters_per_1deg and lat_meters_per_1deg to *taxi_data* computing the length of 1 degree latitude 
  # and longitude for the average latitude between 2 adjacent points (using the equations and parameters above)

  taxi_data$lat_meters_per_1deg <- m1 + m2 * cos(2 * taxi_data$ave_lat*pi/180) + m3 * cos(4 * taxi_data$ave_lat*pi/180) + m4 * cos(6 * taxi_data$ave_lat*pi/180)
  taxi_data$long_meters_per_1deg <- p1 * cos(taxi_data$ave_lat*pi/180) + p2 * cos(3 * taxi_data$ave_lat*pi/180) + p3 * cos(5 * taxi_data$ave_lat*pi/180)

  # convert diff_lat and diff_long from degrees to meters

  taxi_data$diff_long <- taxi_data$diff_long*taxi_data$long_meters_per_1deg
  taxi_data$diff_lat <- taxi_data$diff_lat*taxi_data$lat_meters_per_1deg

  # remove intermediary columns ave_lat, lat_meters_per_1deg, and long_meters_per_1deg 

  taxi_data <- subset.data.frame(taxi_data,select = c("taxi_id","longitude","latitude","begin","new_date_time","interval_sec","diff_lat","diff_long"))

  # Add column distance (in meters) to *taxi_data* by computing Euclidean distance from columns diff_lat and diff_long

  taxi_data$distance <- sqrt(taxi_data$diff_lat^2 + taxi_data$diff_long^2)

  # set distance for the beginning of a new taxi_id to NA

  taxi_data$distance[which(taxi_data$begin == 1)] <- NA

  # remove intermediary columns diff_lat and diff_long 

  taxi_data <- subset.data.frame(taxi_data,select = c("taxi_id","longitude","latitude","begin","new_date_time","interval_sec","distance"))


## figure_dist.R, figure_time.R

This code uses the base R plotting to create the distance and time histograms. 

Note that there were data points beyond the range of the figures (>8000 for distance, >12 for time). These were included in the proportions, even though they were outside the range of the figures. I chose to do this, as I believe this is a more faithful representation of the data.

### figure_dist.R

The steps for figure_distance.R are:

1. Count the number of rows with distance > 8000 (over_8000). 
2. In *t* (a temporary vector equal to taxi_data$distance), set those over_8000 to NA.
3. Draw a density plot (h) to stdout. 
4. The plot h is converted to proportions with the count for over_8000 included, so that the proportions represent those over all the data (including those above 8000).
5. The plot is written to distance.pdf.

In [ ]:
# figure_dist.R

# Count the number of rows with distance > 8000 (over_8000).
over_8000 <- length(which(taxi_data$distance>8000))

# In t (a temporary vector equal to taxi_data$distance), set those over_8000 to NA. 
t <- taxi_data$distance
t[which(t>8000)] <- NA

# Draw a density plot (h) to stdout.
h <- hist(t,xlim=c(0,8000),freq=FALSE,breaks =c(seq(0,8000,by =500)))

# The plot h is converted to proportions with the count for over_8000 included, so that the proportions represent 
# those over all the data (including those above 8000).
h$counts <- h$counts/(sum(h$counts) + over_8000)

# The plot is written to distance.pdf
pdf(file ="distance.pdf")
plot(h,xlab = "distance interval (meters)",ylab = "proportion",main="", col = 34,xaxp  = c(0, 8000, 8),yaxp = c(0,.7,7),tck = 0.01)
dev.off()
rm(t)


### figure_time.R

figure_time.R is similar in construction to figure_distance.R

The steps for figure_time.R are:

1. Set *t* (a temporary vector) equal to taxi_data$interval_sec
2. Convert intervals from sec to min
3. Count the number of rows with interval > 12 min 
4. Set the values over_12 to NA.
5. Draw a density plot (h) to stdout. 
5. The plot h is converted to proportions with the count for over_12 included, so that the proportions represent those over all the data (including those above 12).
5. The plot is written to time.pdf.

In [ ]:
# figure_time.R

# Set *t* (a temporary vector) equal to taxi_data$interval_sec
t <- taxi_data$interval_sec

# convert intervals from sec to min
t <- as.single(t)/60

# Count the number of rows with interval > 12 min 
over_12 <- length(which(t>12))

# Set the values over 12 to NA.
t[which(t>12)] <- NA

# Draw a density plot (h) to stdout. 
h <- hist(t,xlim=c(0,12),freq=FALSE,breaks =c(seq(0,12,by =0.5)))

# The plot h is converted to proportions with the count for over_12 included, 
# so that the proportions represent those over all the data (including those above 12).
h$counts <- h$counts/(sum(h$counts) + over_12)

# The plot is written to time.pdf.
pdf(file ="time.pdf")
plot(h,xlab = "time interval (minutes)",ylab = "proportion",main="", col = 34,xaxp  = c(0, 12, 6),yaxp =c(0,.4,8),tck = 0.01)
dev.off()
rm(t)


## Distance and time figure results

Here is the resulting figure (distance.pdf) from figure_distance.R, and the resulting figure (time.pdf) from figure_time.R:


In [ ]:
library("IRdisplay")
home <- "C:/Users/Steve S/Desktop/startup.ml/taxi/"
display_pdf(file = paste(home,"distance.pdf",sep=""))
display_pdf(file = paste(home,"time.pdf",sep=""))


Note that there are a few differences between these figures and the figures in User_guide_T-drive.pdf. 

First, the above figures use the default for the hist() function, with the range of each bar corresponding to the actual range of values binned for that bar (e.g, for distance, 0-500, 501-1000, etc.). However, the bars in User_guide_T-drive.pdf appears to be centered upon the endpoints of the range (e.g., for distance, 500, 1000, etc.), and the range of the bars (about 400) less than the range for each of the bins (i.e., 500). As the default more accurately represents the data, I chose to keep the default configuration.

Second, while the patterns of data across the figures are similar, the values do not match exactly. This could be due to my use of the proportions over all the data, whereas the figures in the User Guide may have used the proportions for only the values within the range of the figures (e.g., for distance, <= 8000). In particular, the values are higher for the distance proportions in the User Guide. Also, there could be differences due to the use of different samples, with 8890 files and about 10 million data points available online and 10357 files and 15 million data points described in User_guide_T-drive.pdf. 


## spline.R

This code calculates the smoothed latitude and longitude functions separately, both with respect to (wrt) time, using spline(). Three figures were created: <br>
Smoothed Latitude wrt Time (smooth_lat.pdf) <br>
Smoothed Longitude wrt Time (smooth_longt.pdf)<br>
Smoothed Latitude vs. smoothed longitude ((smooth_lat_v_long.pdf) <br>

The code was tested for the first 3 files listed in subdirectory "06", "39.txt", "220.txt", and "234.txt". Figures from "220.txt" are presented below.

Two issues arose in preliminary assessment. First, there were a number of interval durations equal to zero; rows with zero interval durations were removed. The replacement latitude and longitude were the averages across the two rows with the same new_date_time. In practice this seems to make little difference, as inspection of the 3 files found that the latitudes and longitudes were the same across consecutive rows with the same new_date_time. 

Second, there appeared to be some outliers in terms of latitude and longitude. To determine outliers, I used the code written for lat_long_conv.R to create a function to calculate the speed (calc_spped) in meters per second for adjacent points in time. The code found the first instance of a speed above a threshold (mps_limit), removed that point, and then recalculated the speeds. The procedure continued until no rows existed with a speed above the threshold. I chose a relatively conservative (i.e., high) threshold (100 mps = 223.7 mph), as the noise associated with the interval and position values could not be determined.

The steps for spline.R are:

1.	read.csv infile into dataframe *raw*
2.	add column new_date_time to *raw*, converting input date_time to POSIXct
3.	add column new_date_time_m1 to *raw*, with the new_date_time from the previous rows
4.	add column interval_sec to *raw*, set first interval to NA
5.	set *filtered* dataframe equal to *raw*
6.	take out the rows from *filtered* with durations equal to zero <br>
    a.	For the rows one back from rows having an interval of zero, set the latitudes and longitudes equal to the average of the two rows <br>
    b.	remove the rows with interval_sec equal to 0 <br>
7.	remove outliers in speed (above mps_limit) from *filtered*
    a.	set the initial lat, long, int_sec values as input to calc_speed <br>
    b.	calculate the first pass of calc_speed, adding column speed to filtered <br>
    c.	set *s* as a vector containing indices of rows with speeds above thresholds <br>
    d.	while the length of s != 0 (while there are speeds above threshold) <br>
            i.	remove the first instance of speed above threshold
            ii.	recalculate speed 
            iii. re-set s as vector containing indices of rows with speeds above thresholds
8.	print out number of points after each stage of filtering
9.	perform splines upon latitude and longitude (wrt new_date_time)
10.	plot smoothed_latitude wrt time
11.	plot smoothed_longitude wrt time
12.	plot smoothed Latitude vs. Longitude




In [ ]:
# spline.R

# calc_speed returns a vector of speeds in meters/sec.  is essentially the code for lat_long_conv.R, with an added 
# calculation for speed = abs(distance/time) 

calc_speed <- function(lat,long,int_sec) {
  
# add columns lat_m1 and long_m1 in dataframe tmp with the latitides and longitudes of the previous rows
    
  tmp <- data.frame(lat,long,int_sec)
  np <- nrow(tmp)
  t <- tmp$lat[1:(np - 1)]
  t <- c(t[1],t)
  tmp$lat_m1 <- t
  
  t <- tmp$long[1:(np - 1)]
  t <- c(t[1],t)
  tmp$long_m1 <- t

# add columns to tmp for diff_lat, diff_long, and ave_lat
  
  tmp$diff_lat <- abs(tmp$lat - tmp$lat_m1)
  tmp$diff_long <- abs(tmp$long - tmp$long_m1)
  tmp$ave_lat <- (tmp$lat + tmp$lat_m1)/2
  
  # this section uses the equations used by, for example:
  # http://www.csgnetwork.com/degreelenllavcalc.html
  # which is referenced in Wikipedia (https://en.wikipedia.org/wiki/Geographic_coordinate_system)
  # and is apparently accurate to 1 cm per degree latitude/longitude
  
  m1 <- 111132.92     # latitude calculation term 1
  m2 <- -559.82       # latitude calculation term 2
  m3 <- 1.175         # latitude calculation term 3
  m4 <- -0.0023       # latitude calculation term 4
  p1 <- 111412.84     # longitude calculation term 1
  p2 <- -93.5         # longitude calculation term 2
  p3 <- -0.118         # longitude calculation term 3
  
  # add columns lat_meters_per_1deg and long_meters_per_1deg to tmp 
  # these equations give the length of 1 deg (lat and long) in meters for a given latitude, using the ave of the latitudes
  
  tmp$lat_meters_per_1deg <- m1 + m2 * cos(2 * tmp$ave_lat*pi/180) + m3 * cos(4 * tmp$ave_lat*pi/180) + m4 * cos(6 * tmp$ave_lat*pi/180)
  tmp$long_meters_per_1deg <- p1 * cos(tmp$ave_lat*pi/180) + p2 * cos(3 * tmp$ave_lat*pi/180) + p3 * cos(5 * tmp$ave_lat*pi/180)
  
  # now convert differences in degrees to meters
  tmp$diff_long <- tmp$diff_long*tmp$long_meters_per_1deg
  tmp$diff_lat <- tmp$diff_lat*tmp$lat_meters_per_1deg

  # calculate Euclidean distance and speed, m/sec
  tmp$distance <- sqrt(tmp$diff_lat^2 + tmp$diff_long^2)
  tmp$speed <- abs(tmp$distance/as.numeric(tmp$int_sec))
    
  # return the vector for speed  
  ret <- tmp$speed 
  return(ret)
  
}

# beginning of spline 

library(lubridate)
mps_limit <- 100

#home <- "C:/Users/ibshi/Desktop/startup.ml/taxi"
home <- "C:/Users/Steve S/Desktop/startup.ml/taxi"
setwd(home)

# read.csv infile into dataframe raw

#infile <- "/06/39.txt"
infile <- "/06/220.txt"
#infile <- "/06/234.txt"
infile <- paste(home, infile,sep="")

raw <- read.csv(infile, header = FALSE)
colnames(raw) <- c("taxi_id","date_time","longitude","latitude")
npoints_orig <- nrow(raw)

# add column new_date_time to raw, converting input date_time to POSIXct

raw$new_date_time <- ymd_hms(raw$date_time)

# add column new_date_time_m1 to raw, with the new_date_time from the previous rows

t <- raw$new_date_time[1:(npoints_orig-1)]
t <- c(t[1],t)
raw$new_date_time_m1 <- t

# add column interval_sec to raw, set first interval to NA 

raw$interval_sec <- as.duration(interval(raw$new_date_time_m1,raw$new_date_time))
raw$interval_sec[1] <- NA

# set filtered dataframe equal to raw

filtered <- raw

# take out the rows with durations equal to zero

  # For the rows one back from rows having an interval of zero, set the latitudes 
  # and longitudes equal to the average of the two rows

for (k in 2:npoints_orig) {
if (filtered$interval_sec[k] == as.duration(0)){
  filtered$latitude[k-1] <- (filtered$latitude[k-1]+filtered$latitude[k])/2
  filtered$longitude[k-1] <- (filtered$longitude[k-1]+filtered$longitude[k])/2
  }
}

  # remove the rows with interval_sec equal to 0

filtered <- filtered[which(filtered$interval_sec != as.duration(0)),]
npoints_nozero <- nrow(filtered)

# remove outliers in speed (above mps_limit)

  # set the initial lat, long, int_sec values as input to calc_speed

lat <- filtered$latitude
long <- filtered$long
int_sec <- filtered$interval_sec
 
  # calculate the first pass of calc_speed, adding column speed to filtered

filtered$speed <- calc_speed(lat, long, int_sec)

  # set s as a vector containing indices of rows with speeds above thresholds

s <- which(filtered$speed > mps_limit)

  # while the length of s != 0 (while there are speeds above threshold)

while (length(s) != 0) {

  # remove the first instance of speed above threshold
    
  curr_ind <- s[1]
  curr_nrow <- nrow(filtered) 
  filtered <- rbind(filtered[1:(curr_ind-1),],filtered[(curr_ind+1):curr_nrow,])

   # recalculate speed 
    
  lat <- filtered$latitude
  long <- filtered$long
  int_sec <- filtered$interval_sec
  filtered$speed <- calc_speed(lat, long, int_sec)

  # re-set s as vector containing indices of rows with speeds above thresholds
    
  s <- which(filtered$speed > mps_limit)

}

# print out number of points after each stage of filtering

npoints <- nrow(filtered)
print(paste("npoints orig",as.character(npoints_orig)))
print(paste("npoints no zeroes",as.character(npoints_nozero)))
print(paste("npoints removing high speeds",as.character(npoints)))

# perform splines upon latitude and longitude (wrt new_date_time)

smoothed_lat  <- spline(filtered$new_date_time,filtered$latitude,n=length(filtered$new_date_time))
smoothed_long <- spline(filtered$new_date_time,filtered$longitude,n=length(filtered$new_date_time))

# plot smoothed_latitude wrt time

pdf(file = "smooth_lat.pdf")
plot(filtered$new_date_time,filtered$latitude,ylab = "latitude",xlab = "time",main="Smoothed Latitude wrt Time")
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_lat$x,smoothed_lat$y)
dev.off()

# plot smoothed_longitude wrt time 

pdf(file = "smooth_long.pdf")
plot(filtered$new_date_time,filtered$longitude,ylab = "longitude",xlab = "time",main="Smoothed Longitude wrt Time")
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_long$x,smoothed_long$y)
dev.off()

# plot smoothed Latitude vs. Longitude

pdf(file = "smooth_lat_v_long.pdf")
plot(filtered$latitude,filtered$longitude,ylab = "longitude",xlab = "latitude",main = "Smoothed Latitude and Longitude")
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_lat$y,smoothed_long$y)
dev.off()



## figures from spline.R

Below are the results for "~/06/220.txt". For all figures, the points represent the raw data points, and the lines represent the smoothed trajectories.

The output from spline.R was: <br>
[1] "npoints orig 1820" <br>
[1] "npoints no zeroes 1746" <br>
[1] "npoints removing high speeds 1744" <br>
Thus, there were 1820-1746=74 rows with interval duration = 0, and 2 rows with speeds above threshold ( 100 mps)

Here is: <br>
the figure for latitude with respect to (wrt) time (smooth_lat.pdf), <br>
the figure for longitude with respect to (wrt) time (smooth_long.pdf), and <br>
the figure for latitude vs. longitude (smooth_lat_v_long.pdf). <br>


In [ ]:
library("IRdisplay")
home <- "C:/Users/Steve S/Desktop/startup.ml/taxi/"
display_pdf(file = paste(home,"smooth_lat.pdf",sep=""))
display_pdf(file = paste(home,"smooth_long.pdf",sep=""))
display_pdf(file = paste(home,"smooth_lat_v_long.pdf",sep=""))


## spline_display.R

This code does essentially the same thing as spline.R, but displays the figures without saving to a file and waits for a user response. Also, this code displays figures using subsets of the data based upon a range of indices, called beg and end, starting with beg <- 1. In this code the size of the range is set to step_point <- 400

In [ ]:
# spline_display.R

# calc_speed returns a vector of speeds in meters/sec.  is essentially the code for lat_long_conv.R, with an added 
# calculation for speed = abs(distance/time) 

calc_speed <- function(lat,long,int_sec) {
  
# add columns lat_m1 and long_m1 in dataframe tmp with the latitides and longitudes of the previous rows
    
  tmp <- data.frame(lat,long,int_sec)
  np <- nrow(tmp)
  t <- tmp$lat[1:(np - 1)]
  t <- c(t[1],t)
  tmp$lat_m1 <- t
  
  t <- tmp$long[1:(np - 1)]
  t <- c(t[1],t)
  tmp$long_m1 <- t

# add columns to tmp for diff_lat, diff_long, and ave_lat
  
  tmp$diff_lat <- abs(tmp$lat - tmp$lat_m1)
  tmp$diff_long <- abs(tmp$long - tmp$long_m1)
  tmp$ave_lat <- (tmp$lat + tmp$lat_m1)/2
  
  # this section uses the equations used by, for example:
  # http://www.csgnetwork.com/degreelenllavcalc.html
  # which is referenced in Wikipedia (https://en.wikipedia.org/wiki/Geographic_coordinate_system)
  # and is apparently accurate to 1 cm per degree latitude/longitude
  
  m1 <- 111132.92     # latitude calculation term 1
  m2 <- -559.82       # latitude calculation term 2
  m3 <- 1.175         # latitude calculation term 3
  m4 <- -0.0023       # latitude calculation term 4
  p1 <- 111412.84     # longitude calculation term 1
  p2 <- -93.5         # longitude calculation term 2
  p3 <- -0.118         # longitude calculation term 3
  
  # add columns lat_meters_per_1deg and long_meters_per_1deg to tmp 
  # these equations give the length of 1 deg (lat and long) in meters for a given latitude, using the ave of the latitudes
  
  tmp$lat_meters_per_1deg <- m1 + m2 * cos(2 * tmp$ave_lat*pi/180) + m3 * cos(4 * tmp$ave_lat*pi/180) + m4 * cos(6 * tmp$ave_lat*pi/180)
  tmp$long_meters_per_1deg <- p1 * cos(tmp$ave_lat*pi/180) + p2 * cos(3 * tmp$ave_lat*pi/180) + p3 * cos(5 * tmp$ave_lat*pi/180)
  
  # now convert differences in degrees to meters
  tmp$diff_long <- tmp$diff_long*tmp$long_meters_per_1deg
  tmp$diff_lat <- tmp$diff_lat*tmp$lat_meters_per_1deg

  # calculate Euclidean distance and speed, m/sec
  tmp$distance <- sqrt(tmp$diff_lat^2 + tmp$diff_long^2)
  tmp$speed <- abs(tmp$distance/as.numeric(tmp$int_sec))
    
  # return the vector for speed  
  ret <- tmp$speed 
  return(ret)
  
}

# beginning of spline 

library(lubridate)
mps_limit <- 100

#home <- "C:/Users/ibshi/Desktop/startup.ml/taxi"
home <- "C:/Users/Steve S/Desktop/startup.ml/taxi"
setwd(home)

# read.csv infile into dataframe raw

#infile <- "/06/39.txt"
infile <- "/06/220.txt"
#infile <- "/06/234.txt"
infile <- paste(home, infile,sep="")

raw <- read.csv(infile, header = FALSE)
colnames(raw) <- c("taxi_id","date_time","longitude","latitude")
npoints_orig <- nrow(raw)

# add column new_date_time to raw, converting input date_time to POSIXct

raw$new_date_time <- ymd_hms(raw$date_time)

# add column new_date_time_m1 to raw, with the new_date_time from the previous rows

t <- raw$new_date_time[1:(npoints_orig-1)]
t <- c(t[1],t)
raw$new_date_time_m1 <- t

# add column interval_sec to raw, set first interval to NA 

raw$interval_sec <- as.duration(interval(raw$new_date_time_m1,raw$new_date_time))
raw$interval_sec[1] <- NA

# set filtered dataframe equal to raw

filtered <- raw

# take out the rows with durations equal to zero

  # For the rows one back from rows having an interval of zero, set the latitudes 
  # and longitudes equal to the average of the two rows

for (k in 2:npoints_orig) {
if (filtered$interval_sec[k] == as.duration(0)){
  filtered$latitude[k-1] <- (filtered$latitude[k-1]+filtered$latitude[k])/2
  filtered$longitude[k-1] <- (filtered$longitude[k-1]+filtered$longitude[k])/2
  }
}

  # remove the rows with interval_sec equal to 0

filtered <- filtered[which(filtered$interval_sec != as.duration(0)),]
npoints_nozero <- nrow(filtered)

# remove outliers in speed (above mps_limit)

  # set the initial lat, long, int_sec values as input to calc_speed

lat <- filtered$latitude
long <- filtered$long
int_sec <- filtered$interval_sec
 
  # calculate the first pass of calc_speed, adding column speed to filtered

filtered$speed <- calc_speed(lat, long, int_sec)

  # set s as a vector containing indices of rows with speeds above thresholds

s <- which(filtered$speed > mps_limit)

  # while the length of s != 0 (while there are speeds above threshold)

while (length(s) != 0) {

  # remove the first instance of speed above threshold
    
  curr_ind <- s[1]
  curr_nrow <- nrow(filtered) 
  filtered <- rbind(filtered[1:(curr_ind-1),],filtered[(curr_ind+1):curr_nrow,])

   # recalculate speed 
    
  lat <- filtered$latitude
  long <- filtered$long
  int_sec <- filtered$interval_sec
  filtered$speed <- calc_speed(lat, long, int_sec)

  # re-set s as vector containing indices of rows with speeds above thresholds
    
  s <- which(filtered$speed > mps_limit)

}

# print out number of points after each stage of filtering

npoints <- nrow(filtered)
print(paste("npoints orig",as.character(npoints_orig)))
print(paste("npoints no zeroes",as.character(npoints_nozero)))
print(paste("npoints removing high speeds",as.character(npoints)))

# perform splines upon latitude and longitude (wrt new_date_time)

smoothed_lat  <- spline(filtered$new_date_time,filtered$latitude,n=length(filtered$new_date_time))
smoothed_long <- spline(filtered$new_date_time,filtered$longitude,n=length(filtered$new_date_time))

step_point <- 400
num_step <- as.integer(npoints/step_point)

# plot smoothed_latitude wrt time

plot(filtered$new_date_time,filtered$latitude,ylab = "latitude",xlab = "time",main="Smoothed Latitude wrt Time")
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_lat$x,smoothed_lat$y)
cat ("Press [enter] to continue")
line <- readline()

  # plot for ranges, starting with 1 and size of range equal to step_point

for (i in 1:num_step) {
  beg <- step_point*(i - 1)+ 1
  end <- step_point*(i - 1) + step_point
  temp_ti <- paste("Smoothed Latitude wrt Time, points",as.character(beg),"to ", as.character(end))
  plot(filtered$new_date_time[beg:end],filtered$latitude[beg:end],ylab = "latitude",xlab = "time",main=temp_ti)
  mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
  lines(smoothed_lat$x[beg:end],smoothed_lat$y[beg:end])
  cat ("Press [enter] to continue")
  line <- readline()
}

  # plot to the end of the data (npoints)

beg <- step_point*num_step + 1
end <- npoints
temp_ti <- paste("Smoothed Latitude wrt Time, points",as.character(beg),"to ", as.character(end))
plot(filtered$new_date_time[beg:end],filtered$latitude[beg:end],ylab = "latitude",xlab = "time",main=temp_ti)
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_lat$x[beg:end],smoothed_lat$y[beg:end])
cat ("Press [enter] to continue")
line <- readline()

# plot smoothed_longitude wrt time 

plot(filtered$new_date_time,filtered$longitude,ylab = "longitude",xlab = "time",main="Smoothed Longitude wrt Time")
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_long$x,smoothed_long$y)
cat ("Press [enter] to continue")
line <- readline()

  # plot for ranges, starting with 1 and size of range equal to step_point

for (i in 1:num_step) {
    beg <- step_point*(i - 1)+ 1
    end <- step_point*(i - 1) + step_point
    temp_ti <- paste("Smoothed Longitude wrt Time, points",as.character(beg),"to ", as.character(end))
    plot(filtered$new_date_time[beg:end],filtered$longitude[beg:end],ylab = "longitude",xlab = "time",main=temp_ti)
    mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
    lines(smoothed_long$x[beg:end],smoothed_long$y[beg:end])
    cat ("Press [enter] to continue")
    line <- readline()
}

  # plot to the end of the data (npoints}

beg <- step_point*num_step + 1
end <- npoints
temp_ti <- paste("Smoothed Longitude wrt Time, points",as.character(beg),"to ", as.character(end))
plot(filtered$new_date_time[beg:end],filtered$longitude[beg:end],ylab = "longitude",xlab = "time",main=temp_ti)
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_long$x[beg:end],smoothed_long$y[beg:end])
cat ("Press [enter] to continue")
line <- readline()

# plot smoothed Latitude vs. Longitude

plot(filtered$latitude,filtered$longitude,ylab = "longitude",xlab = "latitude",main = "Smoothed Latitude and Longitude")
mtext("Points represent the raw data. Lines represent the smoothed spline trajectory.",side = 3)
lines(smoothed_lat$y,smoothed_long$y)
cat ("Press [enter] to continue")
line <- readline()

  # plot for ranges, starting with 1 and size of range equal to step_point

for (i in 1:num_step) {
    beg <- step_point*(i - 1)+ 1
    end <- step_point*(i - 1) + step_point
    temp_ti <- paste("Smoothed Latitude and Longitude, points",as.character(beg),"to ", as.character(end))
    plot(filtered$latitude[beg:end],filtered$longitude[beg:end],ylab = "longitude",xlab = "latitude",main = temp_ti)
    lines(smoothed_lat$y[beg:end],smoothed_long$y[beg:end])
    cat ("Press [enter] to continue")
    line <- readline()
}

  # plot to the end of the data (npoints}

beg <- step_point*num_step + 1
end <- npoints
temp_ti <- paste("Smoothed Latitude and Longitude, points",as.character(beg),"to ", as.character(end))
plot(filtered$latitude[beg:end],filtered$longitude[beg:end],ylab = "longitude",xlab = "latitude",main = temp_ti)
lines(smoothed_lat$y[beg:end],smoothed_long$y[beg:end])
cat ("Press [enter] to continue")
line <- readline()
